In [1]:
import numpy as np
import tensorflow as tf
import cv2
import keras
from keras.models import load_model
from scipy.misc import imresize
import scipy as sc

C:\Users\btl78\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
my_model = load_model('models/brian2.hdf5')


C:\Users\btl78\Anaconda3\lib\site-packages\keras\models.py:291: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [3]:
img = cv2.imread('../swarm/swarm127.jpg')
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
drone, not_drone = my_model.predict(np.expand_dims(imresize(img,(150,150)), axis = 0))[0]

C:\Users\btl78\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """Entry point for launching an IPython kernel.


In [5]:
drone, not_drone

(1.0, 0.0)

In [6]:
import cv2
import numpy as np

cap = cv2.VideoCapture('../../vid/swarm_drone/swarm_drone.mp4')

while True:
    ret, frame = cap.read()
    if ret == False or cv2.waitKey(1) & 0xff == 27:
        break
    drone, not_drone = my_model.predict(np.expand_dims(imresize(frame,(150,150)), axis = 0))[0]
    cv2.imshow('frame', frame)
    if drone > not_drone:
        print('drone!')
    else:
        print('not drone!')
cv2.destroyAllWindows()
cap.release()

C:\Users\btl78\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  # Remove the CWD from sys.path while we load stuff.


not drone!
drone!
drone!
not drone!
drone!
not drone!
not drone!
not drone!
not drone!
not drone!
drone!
not drone!


In [8]:
def run(func, link='../../vid/swarm_drone/swarm_drone.mp4'):
    cap = cv2.VideoCapture(link)
    i = 0
    while True:
         # Capture frame-by-frame
        ret, frame = cap.read()
        if ret == False or cv2.waitKey(1) & 0xff == 27:
            break
        func(ret, frame)
        i += 30
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        
    cap.release()
    cv2.destroyAllWindows()
def scale(frame, factor=1):
    if factor == 1:
        return frame
    height, wid = frame.shape[:2]
    scaled_wid, scaled_height = np.int(wid * factor), np.int(height * factor)
    frame = cv2.resize(frame, (scaled_wid, scaled_height))
    return frame
def get_orb_kp(frame, orb=cv2.ORB_create()):
    frame = scale(frame)
    kp = orb.detect(frame)
    #kp_drawn = cv2.drawKeypoints(frame, kp, None, color=(0, 255, 0))
    kp_drawn = frame
    return kp, kp_drawn
def kp_to_pairs(kp, dim):
    max_y, max_x, _ = dim
    out = np.zeros((len(kp), 2))
    for i in range(len(kp)):
        x, y = kp[i].pt
        
        out[i] = np.array([x, max_y-y])
    return out
def get_centers(Z, num=10):
    # convert to np.float32
    Z = np.float32(Z)
    # define criteria and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    ret,label,center=cv2.kmeans(Z,num,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
    return center
def plot_boxes(frame, name, centers, dim):
    y_max, x_max, _  = dim
    
    for i in range(len(centers)):
        x, y = centers[i]
        x, y = np.int(x), np.int(y_max - y)
        half_size = 50
        top_left = x - half_size, y - half_size
        bot_right = x + half_size, y + half_size
        crop = frame[y - half_size:y + half_size, x - half_size:x + half_size]
        cv2.imshow('crop', crop)
        drone, not_drone = my_model.predict(np.expand_dims(imresize(crop,(150,150)), axis = 0))[0]
        if drone > .95:
            frame = cv2.rectangle(frame, top_left, bot_right, (0, 255, 0),  1)
        else:
            frame = cv2.rectangle(frame, top_left, bot_right, (0, 0, 255),  1)
        cv2.putText(frame, str(drone),top_left, cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2,cv2.LINE_AA)
        # frame = cv2.circle(frame, (x, y), 50, (0, 255, 0),  1)
    cv2.imshow(name, frame)
def orb_func_boxed(ret, frame):
    kp, frame = get_orb_kp(frame, orb=cv2.ORB_create(nfeatures=1000))
    dim = frame.shape
    Z = kp_to_pairs(kp, dim)
    centers = get_centers(Z, num=3)
    plot_boxes(frame, 'frame', centers, dim)
    
run(orb_func_boxed)

C:\Users\btl78\Anaconda3\lib\site-packages\ipykernel_launcher.py:54: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


NameError: name 'run' is not defined

In [124]:
import cv2
import numpy as np

def run_ex(num, x, y, size):
    cap = cv2.VideoCapture('../../vid/swarm_drone/swarm_drone{}.mp4'.format(num))
    orb = cv2.ORB_create(nfeatures=1000)
    #orb = cv2.ORB_create()
    #cap.set(cv2.CAP_PROP_POS_FRAMES, 1000)
    ret, frame = cap.read()
    scale = 1
    frame = cv2.resize(frame, (0,0), fx=1 / scale, fy=1 / scale) 
    whole = size // scale
    y, x = y // scale, x // scale
    #y, x = 430 // scale, 850 // scale
    crop = frame[y:y + whole, x:x + whole]
    kp_prev, des_prev = orb.detectAndCompute(crop,None)
    cv2.imshow('crop', crop)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


    j = 0
    while True:
        ret, frame = cap.read()
        if ret == False or cv2.waitKey(1) & 0xff == 27:
            break
        frame = cv2.resize(frame, (0,0), fx=1 / scale, fy=1 / scale)
        height,width, _ = frame.shape
        mask = np.zeros((height,width), np.uint8)
        cv2.rectangle(mask, (x,y), (x+whole,y+whole),256,thickness=-1)
        masked = cv2.bitwise_and(frame, frame, mask=mask)
        kp, des = orb.detectAndCompute(masked,None)
        # create BFMatcher object
        bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
        # Match descriptors.
        matches = bf.match(des_prev,des)
        # Sort them in the order of their distance.
        matches = sorted(matches, key = lambda x:x.distance)
        top_matches = matches[:100]
        index = np.mean([x.distance for x in top_matches])
        print(index)
        # Draw first 10 matches.
        out = cv2.drawMatches(crop,kp_prev,masked,kp,top_matches, None, flags=2)
        cv2.imshow('out', out)

        size = len(matches)
        if size > 5:
            if size > 10:
                size = 10
            arr = np.zeros((2,size))
            for i, m in enumerate(matches[:size]):
                x, y = kp[m.trainIdx].pt
                arr[0, i] = x
                arr[1, i] = y
            mean_x, mean_y = np.int(np.mean(arr[0])), np.int(np.mean(arr[1]))
            x = mean_x - whole//2
            y = mean_y - whole//2
        if index < 40:
            cv2.rectangle(frame, (x, y), (x+whole, y+whole), (0,255,0), 1)
        elif index < 65:
            cv2.rectangle(frame, (x, y), (x+whole, y+whole), (0,255,255), 1)
        else:
            cv2.rectangle(frame, (x, y), (x+whole, y+whole), (0,0,255), 1)
            
        if j % 20:
            crop = frame[y:y+whole, x:x+whole]
        
            drone, not_drone = my_model.predict(np.expand_dims(imresize(crop,(150,150)), axis = 0))[0]
            cv2.putText(frame, str(drone),(x,y), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,0),2,cv2.LINE_AA)
        j += 1
        cv2.imshow('frame', frame)
        crop = frame[y:y + whole, x:x + whole]
        kp_prev, des_prev = orb.detectAndCompute(crop,None)
            

    cv2.destroyAllWindows()
    cap.release()

In [125]:
#1
run_ex(num=1, x=550, y=230, size=100)

62.680851063829785
66.51428571428572


C:\Users\btl78\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


63.42
58.89
37.04
76.64
37.0
45.0
32.0
38.0
58.0
39.0
21.666666666666668
62.333333333333336
62.73684210526316
42.82142857142857
26.346153846153847
29.161290322580644
40.5625
55.8
49.05714285714286
29.324324324324323
48.294117647058826
60.689655172413794
65.5925925925926
40.333333333333336
37.96
41.72727272727273
35.6
44.87096774193548
59.392857142857146
63.806451612903224
47.689655172413794
43.17857142857143
32.19047619047619
29.4
39.95
35.333333333333336
27.642857142857142
30.823529411764707
26.923076923076923
33.7
18.444444444444443
21.416666666666668
28.157894736842106
18.25
30.2
21.458333333333332
24.89189189189189
26.676470588235293
33.24390243902439
28.1
24.533333333333335
24.387096774193548
33.375
32.40625
31.942857142857143
31.029411764705884
32.77142857142857
36.12903225806452
36.43333333333333
43.62162162162162
38.142857142857146
36.964285714285715
36.892857142857146
42.8235294117647
47.55555555555556
27.87878787878788
33.0
20.703703703703702
26.805555555555557
28.47222222222

ValueError: tile cannot extend outside image